# Migration der BE-Listen

In [13]:
import openpyxl
import pandas as pd
from natsort import index_natsorted
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
import glob
from datetime import datetime

# alle abzüge werden zusätzlich in einen datumspfad unterhalb des abzugs-pfade geschrieben, damit man ggf. eine historie der geschriebenen dateien rekonstruieren kann


In [2]:
# Einbandart Y
# max Öffnungswinkel AW
# dig mit Begleitung AY
# Testphase BM

def kommentare(bestand):
    wb = openpyxl.load_workbook(f"thomschke/be-{bestand}.xlsm")
    spalten_liste = ("Y", "AW", "AY", "BM")
    for spalte in spalten_liste:
        for cell in wb["Basis"][spalte]:
            if cell.comment:
                wb["Basis"].cell(row=cell.row, column=cell.column+1, value=cell.comment.text.split(":", maxsplit=1)[1])
    
    wb.save(f"thomschke/be-{bestand}-kommentare.xlsx")

In [2]:
def mergen_erstabgleich(bestand):
    gruppen = {"ii":"II", "böink":"Bö Ink", "böm": "Bö M", "schreibmeister":"Schreibmeister", "iii":"III","iv":"IV"}

    be = pd.read_excel(f"thomschke/be-{bestand}-kommentare.xlsx", sheet_name="Basis", dtype={"IDN":"string","AKZ":"string"})
    abzug = pd.read_csv(f"../abzug/{bestand}.csv", dtype={"idn":"string","akz":"string"})

    df = be.merge(abzug, how="outer", left_on=["AKZ", "IDN"], right_on=["akz", "idn"])

    # idn, akz und signatur werden aus abzugs-listen über die BE-Daten kopiert: dabei entsteht kein Datenverlust, weil diese Spalten identisch sein müssen, denn mit ihnen wurden die Daten gemerged
    df.loc[df["IDN"].isna(), "IDN"] = df.idn
    df.loc[df["AKZ"].isna(), "AKZ"] = df.akz
    df.loc[df["Signatur"].isna(), "Signatur"] = df.signatur_a

    # wenn der Datensatz in den Abzugs-Listen war, wird er mit True in der Spalte Digi gekennzeichnet, wenn nicht mit False; dabei bekommen auch Zeilen ein False, für die es noch keinen Datensatz gibt. Beim Durchgang im nächsten Monat wird das automatisch geändert, weil die dann neu angelegten Datensätze in der Abzugs-Liste erscheinen und damit auch hier eingefügt werden.
    df.loc[df["idn"].notna(), "digi"] = True
    df.loc[df["idn"].isna(), "digi"] = False
    df.fillna('', inplace=True)
    df = df.sort_values(by='Signatur', ascending=True, na_position='first', key=lambda X: np.argsort(index_natsorted(df["Signatur"])))
    df["Gruppe"] = gruppen[bestand]
    # die Spalten aus der Abzugs-Liste werden wieder entfernt, die Digi Spalte wird an den Anfang gesetzt
    spaltenliste = list(df.columns)[:-23]
    spaltenliste.insert(0, "digi")
    spaltenliste.insert(0, "Gruppe")
    spaltenliste.insert(4, "bbg")
    spaltenliste.insert(7, "signatur_g")

    wb = openpyxl.load_workbook(f"thomschke/be-{bestand}-kommentare.xlsx")
    wb.remove(wb["Basis"])
    ws = wb.create_sheet("Basis", 0)
    df = df.astype("object")
    for r in dataframe_to_rows(df[spaltenliste], index=False):
        ws.append(r)
    tab = Table(displayName="Basistabelle", ref=ws.dimensions)
    tab.tableStyleInfo = TableStyleInfo(name="Hell1", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=False)
    ws.add_table(tab)
    wb.save(f"listen_fin/{bestand}.xlsx")
    # df.to_excel(f"listen_fin/{bestand}-df.xlsx", columns=spaltenliste, index=False)

In [11]:
def mergen_zweitabgleich(bestand):

    # matching zwischen der kurzschreibweise der bestandsgruppen und der langschreibweise, die in die erste spalte "Gruppe" geschrieben wird.
    gruppen = {"ii":"II", "böink":"Bö Ink", "böm": "Bö M", "schreibmeister":"Schreibmeister", "iii":"III","iv":"IV"}

    #einlesen der aktuellen excel-tabelle des jeweiligen bestands
    be = pd.read_excel(f"listen_fin/{bestand}-recent.xlsx", sheet_name="Basis", dtype={"IDN":"string","AKZ":"string"})
    
    # einlesen des aktuellen cbs-abzugs
    abzug = pd.read_csv(f"../abzug/{bestand}.csv", dtype={"idn":"string","akz":"string"})

    # outer-merge der beiden tabellen, d.h. alle datensätze aus beiden tabellen bleiben erhalten, Gesamttabelle heißt df
    df = be.merge(abzug, how="outer", left_on=["AKZ", "IDN"], right_on=["akz", "idn"])

    df.to_excel("df.xlsx") # nur für bugfixing wird an der stelle die tabelle gespeichert.
    
    # Schreiben der aktualisierten Daten des neuen Abzugs über die alten Daten
    df.loc[df["Signatur"].isna(), "Signatur"] = df.signatur_a_y
    df.loc["bbg_x"] = df.bbg_y
    df.loc["signatur_g_x"] = df.signatur_g_y
    df.loc["signatur_a_x"] = df.signatur_a_y
    df.loc["titel_x"] = df.titel_y
    df.loc["stuecktitel_x"] = df.stuecktitel_y
    df.loc["wert_x"] = df.wert_y
    
    # es gibt spalten, die in beiden tabellen vorkommen, beim mergen versieht pandas deshalb die spalten mit dem suffix _x für die linke tabelle (= die aktuelle gesamttabelle) bzw _y (= CBS-Abzug).
    df.rename({"bbg_x":"bbg","signatur_g_x":"signatur_g", "signatur_a_x":"signatur_a","titel_x":"titel", "stuecktitel_x":"stuecktitel", "wert_x":"wert"}, axis=1, inplace=True)
 
    # alle Datensätze, die nicht im CBS-Abzug waren, werden auf False gesetzt
    df.loc[df["idn"].isna(), "digi"] = False

    # alle Datensätze werden auf True gesetzt, die auch im CBS-abzug waren oder die in der Spalte Whiteliste eine Markierung haben
    df.loc[(df["idn"].notna() | df["Whitelist"].notna()), "digi"] = True

    # alle Datensätze, die Öffnungswinkel 0 haben, werden auf False gesetzt
    df.loc[df["max. Öffnungs-winkel"] == "0", "digi"] = False

    # wenn der Restaurierungsaufwand größer als Null ist, wird das in der Spalte "Restaurierung" mit einem kleinen x markiert. Dazu wird der Datentyp vorher in eine Zahl konvertiert, bisher war es ein string.
    df["Rest.-\nAufwand gesamt\n(in Std.)"] = pd.to_numeric(df["Rest.-\nAufwand gesamt\n(in Std.)"])
    df.loc[df["Rest.-\nAufwand gesamt\n(in Std.)"] > 0, "Restaurierung"] = "x"
    
    # Pandas <NA>-Wert werden durch einen leeren String ersetzt (gibt sonst Probleme beim Schreiben des Tabellenblatts)
    df.fillna('', inplace=True)
    
    # Die Tabelle wird nach der Spalte Signatur mit natürlicher Sortierung sortiert
    df = df.sort_values(by='Signatur', ascending=True, na_position='first', key=lambda X: np.argsort(index_natsorted(df["Signatur"])))
    
    # Spalte Gruppe wird mit aktuellem Bestand gefüllt
    df["Gruppe"] = gruppen[bestand]
    
    # Defragmentierung des DataFrames nach den zahlreichen Einzeländerungen
    df = df.copy()
    
    # link zum portal neu erzeugen
    df["Link zum Portal"] = '=HYPERLINK("https://portal.dnb.de/opac.htm?method=simpleSearch&cqlMode=true&query=idn%3D' + df["IDN"].astype(str) + '", "Portal")'


    # in die liste der zu schreibenden spalten werden alle aufgenommen, außer die des cbs-abzuges, d.h. die letzten 21
    spaltenliste = list(df.columns)[:-21]
    # spaltenliste.remove("bbg")
    # spaltenliste.insert(5, "bbg")
    # spaltenliste.insert(9, "signatur_a")
    # spaltenliste.insert(12, "titel")
    # spaltenliste.insert(13, "stuecktitel")
    # spaltenliste.insert(15, "wert")
    spaltenliste.insert(14, "jahr") if "jahr" in list(df.columns) else None
    # spaltenliste.insert(5, "Link zum Portal")

    
    
    # Das Tabellenblatt "Basis" wird aus der ursprünglichen Excel-Mappe gelöscht.
    wb = openpyxl.load_workbook(f"listen_fin/{bestand}.xlsx")
    wb.remove(wb["Basis"])
    # Ein neues mit gleichem Name wird angelegt, mit den Daten des DataFrames vollgeschrieben
    ws = wb.create_sheet("Basis", 0)
    # Datentypen aller Spalten werden verändert, so dass beim Schreiben in die Exceltabelle keine Verluste auftreten.
    df = df.astype("object")
    # schreiben aller daten in das tabellenblatt
    for r in dataframe_to_rows(df[spaltenliste], index=False):
        ws.append(r)
    #formatieren der tabelle als Tabelle mit Name "Basistabelle"
    tab = Table(displayName="Basistabelle", ref=ws.dimensions)
    ws.add_table(tab)
    # speichern
    wb.save(f"listen_fin/{bestand}-{heute}.xlsx")
    # wb.save(f"listen_fin/{bestand}-recent.xlsx")

In [12]:
heute = datetime.now().strftime("%y-%m-%d-%H%M%S")
bestaende = ["ii", "böink", "böm", "schreibmeister", "iii", "iv"]
# bestaende = ["iv"]


# for bestand in bestaende:
#     kommentare(bestand)

# for bestand in bestaende:
#     mergen_erstabgleich(bestand)

for bestand in bestaende:
    mergen_zweitabgleich(bestand)



/tmp/ipykernel_524/3016455834.py:19: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  df.loc["bbg_x"] = df.bbg_y
/tmp/ipykernel_524/3016455834.py:19: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  df.loc["bbg_x"] = df.bbg_y
/tmp/ipykernel_524/3016455834.py:19: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  df.loc["bb

In [16]:
df = pd.read_excel(f"listen_fin/ii-recent.xlsx", sheet_name="Basis", dtype={"IDN":"string","AKZ":"string"})
df.fillna("", inplace=True)
df

,Gruppe,digi,Restaurierung,Whitelist,Lfd Nr.,Link zum Portal,bbg,AKZ,IDN,signatur_g,...,Risse am Rand,Fehl-stellen im BB\n(groß),Insekten-fraß (stark),Falten / Knicke,saures /\nbrüchiges Papier,Tinten-/ Farbfraß (akut),Register-marken fragil,Klebe-\nstreifen ablösen,Rest.-Aufwand Buchblock (in Std.),Anmerkungen für die Restaurierung am Buchblock
0,II,False,,,,Portal,,,,,...,,,,,,,,,,
1,II,False,,,,Portal,,,,,...,,,,,,,,,,
2,II,True,,,2.0,Portal,Aa,L-1454-315496827,1066966540,"II 1,1 ab - Fragm.",...,,,,,,,,,,
3,II,True,,,3.0,Portal,Aa,L-1459-315498269,1066967997,"II 1,2ba - Fragm.",...,,,,,,,,,,
4,II,True,,,4.0,Portal,Aa,L-1462-31549669X,1066966419,"II 1,2ca - Fragm.",...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,II,False,,,706.0,Portal,,L-1495-375567593,110982050X,,...,,,,,,,,,,
703,II,False,,,707.0,Portal,,L-1489-375567704,111007414X,,...,,,,,,,,,,
704,II,False,,,708.0,Portal,,L-1489-375567712,111007414X,,...,,,,,,,,,,
705,II,False,,,709.0,Portal,,L-1489-375567720,111007414X,,...,,,,,,,,,,


# Probleme

- [x] falsche Signaturen an Fragmenten Böink in alter Liste
- mehrere Exemplare trotz 4105 belegt: Lösung: Thomschke hat diese Bände gesehen. sie werden manuell auf WAHR gesetzt und dann funktioniert es. wenn nicht, müssen sie manuell in die liste gesetzt werden, wobei idn und akz reicht.
- 1141698420 aufgelöster Sammelband, Verknüpfung aus 4105 raus
- Tabellenformatierung übernehmen inkl. Kommentare
- [x] Link zum Portal reparieren
- [x] neue Spalten in Abfrage einfügen
- [x] Beschränkung auf 1000 Zeilen bei Detailansicht erhöhen?
- [-] KEK-Projekt vermerken
- neuen Datensatz mit Zweitabgleich testen
- [x] Öffnungswinkel 0 = FALSCH
- [-] sig_a/g umbenennen in 7109 etc
- [x] 7100, 7109, Signatur, Steht bei, Titel, Stücktitel, jahr
- [x] restaurierungsprojekt = GEsamtaufwand > 0, vorn
- [x] wert
- [x] WAHR auf FALSCH wechsel!!
menschenlesbare Filterbeschreibung ins Wiki
- [x] Notebook in abzug.py integrieren
- [x] länge von df_abzug ermitteln um nur so viele spalten abzuschneiden
- [ ] dvc.yaml anpassen



Erstabgleich bereinigt
- [x] Bö Ink
- [x] Bö M
- [x] II
- [x] III
- [x] IV
- [x] Schreibmeister


Zweitabgleich erfolgreich
- [x] Bö Ink
- [x] Bö M
- [x] II
- [x] III
- [x] Schreibmeister
- [x] IV

